In [1]:
import pandas as pd
import numpy as np
import pymysql
import json

from janome.tokenizer import Tokenizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
    

In [2]:
def connect_sql():

    connection = pymysql.connect(
        host = 'localhost' ,
        user = 'root' ,
        password = 'snskdhkskK123!' ,
        database = 'project_DB'
    )

    query = 'SELECT * FROM ORIG_MRDN_TB'
    df = pd.read_sql(query , connection)

    connection.close()

    return df

In [3]:
def calc_cosine(prompt):
    prompt = [prompt]
    data = connect_sql()
    tokenizer = Tokenizer()
    tokenized_data = [[token.surface for token in tokenizer.tokenize(sentence)] for sentence in prompt]
    w2v_model = Word2Vec(sentences = tokenized_data , vector_size = 100 , window = 5 , min_count = 1 ,
                         workers = 4)
    
    sentence_embeddings = [
        sum([w2v_model.wv[word] for word in sentence if word in w2v_model.wv]) / len(sentence)
        for sentence in tokenized_data
    ]

    similarity_scores = []
    for idx in range(data.shape[0]):
        similarity_score = cosine_similarity(sentence_embeddings , [json.loads(data.loc[idx , 'EMBEDDED_INFORMATION'])])
        similarity_scores.append((idx , similarity_score))
    return similarity_scores

In [4]:
similarity_data = calc_cosine('肩こり')

/var/folders/cg/w5448dgs6vgg_kqyxn1mmxd80000gn/T/ipykernel_42689/2289350848.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query , connection)


In [22]:
sorted(similarity_data , reverse = True , key = lambda x : x[1][0][0])

[(122, array([[0.29790453]])),
 (134, array([[0.29699134]])),
 (312, array([[0.29693865]])),
 (270, array([[0.29634497]])),
 (112, array([[0.29621607]])),
 (1, array([[0.29611794]])),
 (339, array([[0.29607153]])),
 (108, array([[0.2957249]])),
 (17, array([[0.29566625]])),
 (6, array([[0.29552168]])),
 (42, array([[0.29546058]])),
 (18, array([[0.29539406]])),
 (131, array([[0.29537589]])),
 (124, array([[0.29534759]])),
 (110, array([[0.29529509]])),
 (113, array([[0.29514645]])),
 (340, array([[0.29512212]])),
 (128, array([[0.29507319]])),
 (19, array([[0.29500848]])),
 (102, array([[0.29494216]])),
 (291, array([[0.29490586]])),
 (269, array([[0.29489821]])),
 (194, array([[0.29489277]])),
 (353, array([[0.294728]])),
 (192, array([[0.29471483]])),
 (358, array([[0.29468073]])),
 (104, array([[0.29468066]])),
 (321, array([[0.29466045]])),
 (133, array([[0.29463122]])),
 (311, array([[0.29461585]])),
 (262, array([[0.29455803]])),
 (132, array([[0.29454413]])),
 (319, array([[0.29